In [0]:
import pyspark

In [0]:
# import the dataset
df_fake = sqlContext.sql("SELECT text FROM fake_3_csv")
df_true = sqlContext.sql("SELECT text FROM true_2_csv")

In [0]:
# add labels
from pyspark.sql.functions import lit
df_true = df_true.withColumn('label',lit(0))
df_fake = df_fake.withColumn('label',lit(1))

In [0]:
# properties of data
df_true.describe().show()
df_fake.describe().show()

+-------+--------------------+-----+
summary| text|label|
+-------+--------------------+-----+
 count| 21417|21417|
 mean| null| 0.0|
 stddev| null| 0.0|
 min| | 0|
 max|neVIENNA (Reuters...| 0|
+-------+--------------------+-----+

+-------+--------------------+-----+
summary| text|label|
+-------+--------------------+-----+
 count| 23481|23489|
 mean| null| 1.0|
 stddev| null| 0.0|
 min| | 1|
 max|youngers these da...| 1|
+-------+--------------------+-----+

In [0]:
# remove duplicates
df_fake = df_fake.drop_duplicates()
df_true = df_true.drop_duplicates()
df_fake.describe().show()
df_true.describe().show()

+-------+--------------------+-----+
summary| text|label|
+-------+--------------------+-----+
 count| 17455|17456|
 mean| null| 1.0|
 stddev| null| 0.0|
 min| | 1|
 max|youngers these da...| 1|
+-------+--------------------+-----+

+-------+--------------------+-----+
summary| text|label|
+-------+--------------------+-----+
 count| 21192|21192|
 mean| null| 0.0|
 stddev| null| 0.0|
 min| | 0|
 max|neVIENNA (Reuters...| 0|
+-------+--------------------+-----+

In [0]:
# drop null values
df_fake = df_fake.dropna()
df_fake.describe().show()
df_true = df_true.dropna()
df_true.describe().show()

+-------+--------------------+-----+
summary| text|label|
+-------+--------------------+-----+
 count| 17455|17455|
 mean| null| 1.0|
 stddev| null| 0.0|
 min| | 1|
 max|youngers these da...| 1|
+-------+--------------------+-----+

+-------+--------------------+-----+
summary| text|label|
+-------+--------------------+-----+
 count| 21192|21192|
 mean| null| 0.0|
 stddev| null| 0.0|
 min| | 0|
 max|neVIENNA (Reuters...| 0|
+-------+--------------------+-----+

In [0]:
# balance the imbalanced part 
df_true = df_true.sample(fraction=17455/21192)
df_true.count()

Out[8]: 17533

In [0]:
# merge entire data
df = df_true.union(df_fake)

In [0]:
df.show()

+--------------------+-----+
 text|label|
+--------------------+-----+
WASHINGTON (Reute...| 0|
WASHINGTON (Reute...| 0|
WASHINGTON (Reute...| 0|
BEIJING (Reuters)...| 0|
WASHINGTON (Reute...| 0|
HOUSTON (Reuters)...| 0|
(Reuters) - Apple...| 0|
WASHINGTON (Reute...| 0|
MOSCOW (Reuters) ...| 0|
WASHINGTON (Reute...| 0|
WASHINGTON (Reute...| 0|
WASHINGTON (Reute...| 0|
WASHINGTON (Reute...| 0|
FRANKFURT (Reuter...| 0|
WASHINGTON (Reute...| 0|
The White House o...| 0|
HAVANA (Reuters) ...| 0|
SEATTLE (Reuters)...| 0|
WASHINGTON (Reute...| 0|
(Reuters) - The U...| 0|
+--------------------+-----+
only showing top 20 rows

In [0]:
df.describe().show()

+-------+--------------------+------------------+
summary| text| label|
+-------+--------------------+------------------+
 count| 34988| 34988|
 mean| null|0.4988853321138676|
 stddev| null| 0.500005902956336|
 min| | 0|
 max|youngers these da...| 1|
+-------+--------------------+------------------+

In [0]:
# text preprocessing
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

regex_tokenizer = RegexTokenizer(inputCol="text", outputCol="token_text", pattern="\\W")
stop_word_remover = StopWordsRemover(inputCol='token_text',outputCol='cleaned_tokens')
count_vec = CountVectorizer(inputCol='cleaned_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")

In [0]:
# import necessary module
from pyspark.ml import Pipeline
data_prep_pipe = Pipeline(stages=[regex_tokenizer,stop_word_remover,count_vec,idf])

from pyspark.ml.evaluation import MulticlassClassificationEvaluator , BinaryClassificationEvaluator
acc_eval = MulticlassClassificationEvaluator()
bin_score = BinaryClassificationEvaluator()

In [0]:
# tf-idf encoding
data_gen = data_prep_pipe.fit(df)
transformed_df = data_gen.transform(df)
data = transformed_df.withColumnRenamed('tf_idf','features').select(['label','features'])
data.show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0|(118552,[3,8,9,11...|
 0|(118552,[0,1,2,3,...|
 0|(118552,[0,1,2,9,...|
 0|(118552,[0,1,2,3,...|
 0|(118552,[1,3,4,9,...|
 0|(118552,[0,1,2,3,...|
 0|(118552,[0,1,2,4,...|
 0|(118552,[0,1,2,3,...|
 0|(118552,[1,3,6,9,...|
 0|(118552,[0,1,2,3,...|
 0|(118552,[0,1,2,3,...|
 0|(118552,[1,3,9,12...|
 0|(118552,[0,1,2,3,...|
 0|(118552,[0,1,2,3,...|
 0|(118552,[0,1,2,3,...|
 0|(118552,[0,1,2,3,...|
 0|(118552,[0,1,2,3,...|
 0|(118552,[0,1,2,3,...|
 0|(118552,[1,3,9,11...|
 0|(118552,[0,1,2,3,...|
+-----+--------------------+
only showing top 20 rows

In [0]:
# split the data to train and test
train_df , test_df = data.randomSplit([0.7,0.3])
print(f'train_df size is {train_df.count()}')
print(f'test_df size is {test_df.count()}')
train_df.groupBy('label').count().show()

train_df size is 24523
test_df size is 10465
+-----+-----+
label|count|
+-----+-----+
 1|12212|
 0|12311|
+-----+-----+

In [0]:
# classification with Naive Bayes
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes()
fake_news_classifier_nb = nb.fit(train_df)
fake_news_classifier_nb_pred = fake_news_classifier_nb.transform(test_df)
fake_news_classifier_nb_pred.show()

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0|(118552,[0,1,2,3,...|[-15437.776572822...|[1.0,1.6355258046...| 0.0|
 0|(118552,[0,1,2,3,...|[-12971.394955556...| [1.0,0.0]| 0.0|
 0|(118552,[0,1,2,3,...|[-8581.3424840808...|[1.0,2.6273240222...| 0.0|
 0|(118552,[0,1,2,3,...|[-11020.062255598...|[1.0,1.8692750746...| 0.0|
 0|(118552,[0,1,2,3,...|[-9666.5357322017...|[1.0,1.0626253099...| 0.0|
 0|(118552,[0,1,2,3,...|[-5015.9264711630...|[1.0,3.7185315349...| 0.0|
 0|(118552,[0,1,2,3,...|[-8236.9068842912...|[1.0,4.5616821375...| 0.0|
 0|(118552,[0,1,2,3,...|[-1499.0834511187...|[6.69528474163613...| 1.0|
 0|(118552,[0,1,2,4,...|[-7263.6383553161...|[0.14998916845081...| 1.0|
 0|(118552,[0,1,2,9,...|[-1068.1335872747...|[1.0,4.4924775532...| 0.0|
 0|(118552,[0,1,3,4,...|[-17160.396941488...|[1.51123256922680...| 1.0|
 0|(118552,[0,1,7,8,...|[-5639.2985163241...|[6.24903650039116...| 1.0|
 0|(118552,[1,2,3,8,...|[-8130.8535804222...| [1.0,0.0]| 0.0|
 0|(118552,[1,2,6,7,...|[-7994.9788149445...| [1.0,0.0]| 0.0|
 0|(118552,[1,2,7,9,...|[-11353.577296659...| [1.0,0.0]| 0.0|
 0|(118552,[1,2,9,12...|[-1652.9947104666...|[1.0,1.2556919597...| 0.0|
 0|(118552,[1,2,9,17...|[-1579.2811752278...|[1.0,1.0929840426...| 0.0|
 0|(118552,[1,3,4,5,...|[-16690.727951767...| [1.0,0.0]| 0.0|
 0|(118552,[1,3,4,9,...|[-1221.2655005723...|[5.34968920371000...| 1.0|
 0|(118552,[1,3,5,7,...|[-7530.5915669112...|[1.0,2.3803866470...| 0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [0]:
# model evaluation
acc_nb = acc_eval.evaluate(fake_news_classifier_nb_pred)
print("Accuracy of Naive Bayes model at predicting fake news is: {}".format(acc_nb))

Accuracy of Naive Bayes model at predicting fake news is: 0.9399904815990558

In [0]:
fake_news_classifier_nb_pred.groupBy('label').count().show()
fake_news_classifier_nb_pred.groupBy('prediction').count().show()

+-----+-----+
label|count|
+-----+-----+
 1| 5243|
 0| 5222|
+-----+-----+

+----------+-----+
prediction|count|
+----------+-----+
 0.0| 5226|
 1.0| 5239|
+----------+-----+

In [0]:
# classification with Linear SVC
from pyspark.ml.classification import LinearSVC
svc = LinearSVC()
fake_news_classifier_svc = svc.fit(train_df)
fake_news_classifier_svc_pred = fake_news_classifier_svc.transform(test_df)
acc_svc = acc_eval.evaluate(fake_news_classifier_svc_pred)
bin_score_svc = bin_score.evaluate(fake_news_classifier_svc_pred)
print("Accuracy of Linear SVC model at predicting fake news is: {} and area under curve is {}".format(acc_svc,bin_score_svc))

Accuracy of Linear SVC model at predicting fake news is: 0.9888198398416259 and area under curve is 0.9976452709319046

In [0]:
# classification with Logistic Regression
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression()
fake_news_classifier_lr = lr.fit(train_df)
fake_news_classifier_lr_pred = fake_news_classifier_lr.transform(test_df)
acc_lr = acc_eval.evaluate(fake_news_classifier_lr_pred)
bin_score_lr = bin_score.evaluate(fake_news_classifier_lr_pred)
print("Accuracy of Logistic Regression model at predicting fake news is: {} and area under curve is {}".format(acc_lr,bin_score_lr))

Accuracy of Logistic Regression model at predicting fake news is: 0.9608061194346611 and area under curve is 0.9904983559264852